In [33]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Initialize Chrome options
chrome_options = Options()

# Set up date formatting for today's date
today = datetime.today().strftime('%Y-%m-%d')


In [2]:
#Url of the Linkedin Page you want to scrape
page = 'https://www.linkedin.com/company/airesearchcentre/'

In [25]:
#access Webriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

email = "woxsenailab@gmail.com"
password = "Ai@l@bfaculty@2024"
csv_file = "social_posts.csv"

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")

browser = webdriver.Chrome(options=options)
        
    
print("Logging into LinkedIn")
browser.get("https://linkedin.com/uas/login")

# Wait for login elements to load
wait = WebDriverWait(browser, 10)
username = wait.until(EC.presence_of_element_located((By.ID, "username")))
username.send_keys(email)

pword = browser.find_element(By.ID, "password")
pword.send_keys(password)
browser.find_element(By.XPATH, "//button[@type='submit']").click()

Logging into LinkedIn


In [143]:
#Go to webpage
browser.get("https://www.linkedin.com/feed/update/urn:li:activity:7353310347857969153/?collapsed=1")
html = browser.page_source
soup =bs(html, "html.parser")
post = soup.find_all("div", class_="fie-impression-container")

In [118]:
dir(soup.find("span", class_="visually-hidden"))

['EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'MAIN_CONTENT_STRING_TYPES',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '_TreeTraversalEvent',
 '__annotations__',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_event_stream',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_insert',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_self_and',
 '_should_pretty_print',
 'append',
 'attribute_value_list_class',
 '

In [159]:
post[0]

<div class="fie-impression-container">
<!-- --> <div class="relative">
<div class="qzuFAMviCunwZodJszLGpUGGuifyEReY display-flex align-items-flex-start update-components-actor--with-control-menu">
<!-- -->
<div class="update-components-actor__container display-flex flex-grow-1">
<a aria-label="Centre for Languages and Multicultural Communication (CLMC) - Woxsen University, graphic." class="DGvtWMHHFxUFucvarCuTmPvyMuWuwlaQNuYA update-components-actor__image relative" data-test-app-aware-link="" href="https://www.linkedin.com/showcase/centre-for-languages-and-multicultural-communication-woxsen-university/posts" target="_self">
<span class="js-update-components-actor__avatar">
<div class="ivm-image-view-model update-components-actor__avatar">
<div class="ivm-view-attr__img-wrapper">
<!-- -->
<!-- --> <img alt="" class="ivm-view-attr__img--centered EntityPhoto-square-3 update-components-actor__avatar-image evi-image lazy-image ember-view" height="48" id="ember34" loading="lazy" src="https:

In [157]:
post = soup.find_all("div", class_="fie-impression-container")
with open('sample_linkedin_post.html', 'w+') as f:
    f.write(post[0].string())

TypeError: 'NoneType' object is not callable

In [57]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os

def init_driver(headless=True):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--log-level=3")
    return webdriver.Chrome(service=Service(), options=chrome_options)

def extract_post_details(driver, url):
    driver.get(url)
    time.sleep(5)  # wait for JS to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    post_data = {
        "url": url,
        "author": None,
        "content": None,
        "image": None,
        "timestamp": None,
        "reactions": None
    }

    # Try multiple fallback selectors as LinkedIn frequently changes class names
    try:
        post_data["author"] = soup.find("div", class_="qzuFAMviCunwZodJszLGpUGGuifyEReY").get_text(strip=True)
    except:
        pass

    try:
        content_div = soup.find("div", {"class": lambda x: x and "update-components-text" in x})
        if content_div:
            post_data["content"] = content_div.get_text(separator="\n", strip=True)
    except:
        pass

    try:
        img_tag = soup.find("img", {"src": True, "class": lambda x: x and "update-components-image" in x})
        if img_tag:
            post_data["image"] = img_tag["src"]
    except:
        pass

    try:
        time_tag = soup.find("span", class_="visually-hidden")
        if time_tag:
            post_data["timestamp"] = time_tag.get_text(strip=True)
    except:
        pass

    try:
        reactions_span = soup.find("span", class_="social-details-social-counts__reactions-count")
        post_data["reactions"] = reactions_span.get_text(strip=True) if reactions_span else None
    except:
        pass

    return post_data

def generate_html(posts, output_file="linkedin_posts.html"):
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("<html><head><title>LinkedIn Posts</title></head><body>")
        for post in posts:
            f.write(f"<div style='border:1px solid #ccc;padding:10px;margin:10px'>")
            f.write(f"<p><strong>Author:</strong> {post['author']}</p>")
            f.write(f"<p><strong>Time:</strong> {post['timestamp']}</p>")
            f.write(f"<p><strong>Reactions:</strong> {post['reactions']}</p>")
            f.write(f"<p><strong>Content:</strong><br>{post['content']}</p>")
            if post["image"]:
                f.write(f"<img src='{post['image']}' style='max-width:400px'><br>")
            f.write(f"<a href='{post['url']}' target='_blank'>View on LinkedIn</a>")
            f.write("</div>")
        f.write("</body></html>")

def main():
    # Replace with your actual list of post URLs
    # urls = pd.read_csv("linkedin_urls.csv")["url"].tolist()
    urls = ['https://www.linkedin.com/feed/update/urn:li:activity:7353310347857969153?collapsed=1']

    driver = init_driver()
    all_posts = []

    for url in urls:
        print(f"Scraping: {url}")
        try:
            post = extract_post_details(driver, url)
            all_posts.append(post)
            print(all_posts)
        except Exception as e:
            print(f"Error scraping {url}: {e}")

    driver.quit()
    generate_html(all_posts)
    print(f"✅ Scraping complete. HTML output saved to 'linkedin_posts.html'")

if __name__ == "__main__":
    main()


Scraping: https://www.linkedin.com/feed/update/urn:li:activity:7353310347857969153?collapsed=1
[{'url': 'https://www.linkedin.com/feed/update/urn:li:activity:7353310347857969153?collapsed=1', 'author': None, 'content': None, 'image': None, 'timestamp': None, 'reactions': None}]
✅ Scraping complete. HTML output saved to 'linkedin_posts.html'


In [59]:
soup.find("div", class_="qzuFAMviCunwZodJszLGpUGGuifyEReY").get_text(strip=True)

'Centre for Languages and Multicultural Communication (CLMC) - Woxsen UniversityCentre for Languages and Multicultural Communication (CLMC) - Woxsen University212 followers212 followers5d •5 days ago • Visible to anyone on or off LinkedInFollow'